In [1]:
import time
import tensorflow as tf
import sklearn.utils
import sklearn.preprocessing
import datetime
import numpy as np

import os

def make_generator(nsteps):
    generator = tf.keras.Sequential()
    generator.add(tf.keras.layers.Dense(5, input_shape=(5,), activation='relu')) # 5
    generator.add(tf.keras.layers.BatchNormalization())
    generator.add(tf.keras.layers.Dense(10, activation='relu'))                  # 10
    generator.add(tf.keras.layers.BatchNormalization())
    generator.add(tf.keras.layers.Dense((5*nsteps), activation='relu'))          # 25
    generator.add(tf.keras.layers.BatchNormalization())
    generator.add(tf.keras.layers.Dense((5*nsteps), activation='tanh'))          # 25

    return generator


def make_critic(nsteps):
    discriminator = tf.keras.Sequential()
    discriminator.add(tf.keras.layers.Dense(5*nsteps, input_shape=(5*nsteps,)))
    discriminator.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    discriminator.add(tf.keras.layers.Dropout(0.3))
    discriminator.add(tf.keras.layers.Dense(10))
    discriminator.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    discriminator.add(tf.keras.layers.Dense(5))
    discriminator.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    discriminator.add(tf.keras.layers.Dropout(0.3))
    discriminator.add(tf.keras.layers.Flatten())
    discriminator.add(tf.keras.layers.Dense(1))


    return discriminator


def discriminator_loss(d_real, d_fake):
    d_loss = tf.reduce_mean(d_fake) - tf.reduce_mean(d_real)
    return d_loss

def generator_loss(d_fake):
    g_loss = -tf.reduce_mean(d_fake)
    return g_loss

@tf.function
def train_step(noise, real, lmbda, n_critic, g, c, g_opt, c_opt, g1_loss, d1_loss, w_loss):   
    batch_size = len(real)

    ##### train critic ######
    
    #print('inside train step') 

    for i in range(n_critic):    
        with tf.GradientTape() as t:
            with tf.GradientTape() as t1:
                fake = g(noise, training=True) # training=False?

                epsilon = tf.random.uniform(shape=[batch_size, 1], minval=0., maxval=1.)
                #print ('types', real.dtype, epsilon.dtype, fake.dtype)                
                interpolated = real + epsilon * (fake - real) 
                t1.watch(interpolated)
                c_inter = c(interpolated, training=True)  
                d_real = c(real, training=True)
                d_fake = c(fake, training=True)
                d_loss = discriminator_loss(d_real, d_fake)   #initial c loss
                
                #print('d loop', d_real.shape,  d_fake.shape,  d_loss.shape, c_inter.shape) # batch_size by 1 except for d_loss
                #print('d loop',  interpolated.shape, inpt.shape, real.shape, fake.shape) # 
                
                
                #print('discriminator loop',i ,'---------------------------------------------')    
                #print('min and max values of fake',np.min(fake.numpy()),np.max(fake.numpy()))
                #print('min and max values of real',np.min(real),np.max(real) )
                #print('min and max values of d_fake',np.min(d_fake.numpy()),np.max(d_fake.numpy()))
                #print('min and max values of d_real',np.min(d_real.numpy()),np.max(d_real.numpy()))
                
            grad_interpolated = t1.gradient(c_inter, interpolated)
            
            #print('interpolated      ', interpolated.numpy())
            #print('c_inter           ', c_inter.numpy())
            #print('grad_interpolated', grad_interpolated.numpy())
            
            #print('grad_interpolated itself ', grad_interpolated.numpy().shape) # batch_size by 25
            #print('grad_interpolated square ', tf.square(grad_interpolated).numpy().shape) # batch_size by 25
            #print('grad_interpolated red sum', tf.reduce_sum(tf.square(grad_interpolated), axis=[1]).numpy().shape) # batch_size by 1
            #print('grad_interpolated sqrt', tf.sqrt(tf.reduce_sum(tf.square(grad_interpolated), axis=[1])).numpy().shape) # batch_size by 1            
            
                     #tf.sqrt(tf.reduce_sum(tf.square(x)) + 1.0e-12)
            slopes = tf.sqrt(tf.reduce_sum(tf.square(grad_interpolated) + 1e-12, axis=[1])) # 
            
            gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)
            #print('slopes, grad penalty', slopes.numpy(), gradient_penalty.numpy())

            new_d_loss = d_loss + (lmbda*gradient_penalty)  #new c loss
            #print('d_loss and new_d_loss',d_loss.numpy(), new_d_loss.numpy()) 
        
        c_grad = t.gradient(new_d_loss, c.trainable_variables)
        #print('length and type c_grad', len(c_grad), type(c_grad))  # length of c_grad was 8? type list
        #print('length and type c_grad[0]', (c_grad[0].shape), type(c_grad[0]))  # length of c_grad was 8? type list
        #print('values of cgrad',c_grad)
        #print('c.trainable_variables', c.trainable_variables)
        c_opt.apply_gradients(zip(c_grad, c.trainable_variables))


    ##### train generator #####

    with tf.GradientTape() as gen_tape:
        fake_images = g(noise, training=True)
        d_fake = c(fake_images, training=True) # training=False?
        g_loss = generator_loss(d_fake)

    gen_grads = gen_tape.gradient(g_loss, g.trainable_variables)
    g_opt.apply_gradients(zip(gen_grads, g.trainable_variables))
    
    
    #print('g.trainable_variables')
    #for v in g.trainable_variables:
    #  print(v.name)
    #print('c.trainable_variables')
    #for v in c.trainable_variables:
    #  print(v.name)

    ### for tensorboard
    g1_loss(g_loss)
    d1_loss(new_d_loss)
    w_loss((-1)*(d_loss))  #wasserstein distance


    return 

def train(nsteps,ndims,lmbda,n_critic,batch_size,batches,training_data,input_to_GAN, epochs, g, c, g_opt, c_opt, g1_loss, d1_loss, w_loss, g1_summary_writer, d1_summary_writer, w_summary_writer):

    losses = np.zeros((epochs,4))

    for epoch in range(epochs):

        noise = input_to_GAN
        real_data = training_data #X1.astype('int')

        # uncommenting this line means that the noise is not paired with the outputs (probably desirable)
        #noise = np.random.normal(size=[noise.shape[0],noise.shape[1]])
 
        real_data, noise = sklearn.utils.shuffle(real_data, noise) #shuffle each epoch
        #print ('shuffled l_input1 and xn1_comp')
        
        xx1 = real_data.reshape(batches, batch_size, ndims*nsteps)
        inpt1 = noise.reshape(batches, batch_size, ndims)
        #print ('data arranged in batches')
        
        print(g.layers[0].weights)

        for i in range(len(xx1)):
            #print('calling train_step', i ,'of',len(xx1), '-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*')  
            train_step(inpt1[i], xx1[i], lmbda, n_critic, g, c, g_opt, c_opt, g1_loss, d1_loss, w_loss)
            #print('back from train_step')  

        print('epoch:', epoch, '*************************************************')    
        print('gen loss', g1_loss.result().numpy(), 'd loss', d1_loss.result().numpy(), 'w_loss' , w_loss.result().numpy())

        losses[epoch,:] = [ epoch+1, g1_loss.result().numpy() ,   d1_loss.result().numpy(),  w_loss.result().numpy()]

        with g1_summary_writer.as_default():
            tf.summary.scalar('loss', g1_loss.result(), step=epoch)

        with d1_summary_writer.as_default():
            tf.summary.scalar('loss', d1_loss.result(), step=epoch)

        with w_summary_writer.as_default():
            tf.summary.scalar('loss', w_loss.result(), step=epoch)
            
        #print('reset states')
        g1_loss.reset_states()
        d1_loss.reset_states()
        w_loss.reset_states()

        if (epoch + 1) % 1000 == 0:
        #if epoch < 100 or (epoch + 1) % 100 == 0 :
                       
            saved_g1_dir = './saved_g_' + str(epoch + 1)
            saved_d1_dir = './saved_c_' + str(epoch + 1)
            tf.keras.models.save_model(g, saved_g1_dir)
            tf.keras.models.save_model(c, saved_d1_dir)

    np.savetxt('losses.csv', losses, delimiter=',')

    return g


def learn_hypersurface_from_POD_coeffs(input_to_GAN, training_data, nsteps, nPOD, ndims, lmbda, n_critic, batch_size, batches, ndims_latent_input):
    # nPOD not needed

    try:
      print('looking for previous saved models')
      saved_g1_dir = './saved_g_' + str(model_number)
      g = tf.keras.models.load_model(saved_g1_dir)

      saved_d1_dir = './saved_c_' + str(model_number)
      c = tf.keras.models.load_model(saved_d1_dir)


    except:
      print('making new generator and critic')
      g = make_generator(nsteps)
      c = make_critic(nsteps)


    g_opt = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0, beta_2=0.9)
    c_opt = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0, beta_2=0.9)

    g1_loss = tf.keras.metrics.Mean('g1_loss', dtype=tf.float32)
    d1_loss = tf.keras.metrics.Mean('d1_loss', dtype=tf.float32)
    w_loss = tf.keras.metrics.Mean('w_loss', dtype=tf.float32)

    # logs to follow losses on tensorboard
    print('initialising logs for tensorboard')
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    g1_log_dir = './logs/gradient_tape/' + current_time + '/g'
    d1_log_dir = './logs/gradient_tape/' + current_time + '/d'
    w_log_dir = './logs/gradient_tape/' + current_time + '/w'

    g1_summary_writer = tf.summary.create_file_writer(g1_log_dir)
    d1_summary_writer = tf.summary.create_file_writer(d1_log_dir)
    w_summary_writer = tf.summary.create_file_writer(w_log_dir)


    print('beginning training')
    epochs = 500
    generator = train(nsteps,ndims,lmbda,n_critic,batch_size,batches,training_data,input_to_GAN, epochs, g, c, g_opt, c_opt, g1_loss, d1_loss, w_loss, g1_summary_writer, d1_summary_writer, w_summary_writer)
    print('ending training')


    # generate some random inputs and put through generator
    number_test_examples = 10
    test_input = tf.random.normal([number_test_examples, ndims_latent_input])
    predictions = generator(test_input, training=False)
    #predictions = generator.predict(test_input) # number_test_examples by ndims_latent_input

    predictions_np = predictions.numpy() # nExamples by nPOD*nsteps
    #tf.compat.v1.InteractiveSession()
    #predictions_np = predictions.numpy().
    print('Shape of the output of the GAN', predictions_np.shape)
    predictions_np = predictions_np.reshape(number_test_examples*nsteps, nPOD)
    print('Reshaping the GAN output (in order to apply inverse scaling)', predictions_np.shape)

    return predictions_np, generator



In [2]:

# -----------------------------------------------------------------------------------------------------------------------------
# reproducibility
import random

np.random.seed(143)
random.seed(143)
tf.random.set_seed(143)

# read in data, reshape and normalise
lmbda = 10
n_critic = 5

batch_size = 20 # 32  
batches = 10    # 900 

ndims_latent_input = 5 # 128 # latent variables for GAN

# data settings
nsteps = 5  #number of consecutive timesteps in gan
ndims = 5 # == nPOD # 128 # reduced variables i.e. POD coefficients or AE latent variables

# reading in the data
print('Reading in the POD coeffs.')
#csv_data = np.loadtxt('/kaggle/input/fpc-204examples-5pod-without-ic/POD_coeffs_1_204.csv', delimiter=',')
csv_data = np.loadtxt('../data/processed/POD_coeffs_1_204_orig.csv', delimiter=',')
csv_data = np.float32(csv_data)
print('type and shape (nPOD by nTrain) of POD coeffs from csv file', type(csv_data), csv_data.shape, csv_data.dtype)

nTrain = csv_data.shape[1]
nPOD = csv_data.shape[0]

csv_data = csv_data.T # nTrain by nPOD

# scaling the POD coeffs
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=[-1,1])
print('shape csv data for the scaling', csv_data.shape) 
csv_data = scaling.fit_transform(csv_data)

# check that the columns are scaled between min and max values (-1,1)
for icol in range(csv_data.shape[1]):
    print('min and max of col, ', icol ,' of csv_data:', np.min(csv_data[:,icol]), np.max(csv_data[:,icol]) )

# create nsteps time levels for the training_data for the GAN
t_begin = 0
t_end = nTrain - nsteps + 1
training_data = np.zeros((t_end,nPOD*nsteps),dtype=np.float32) # nTrain by nsteps*nPOD # 'float32' or np.float32

for step in range(nsteps):
    #print ('training data - cols',step*nPOD,'to',(step+1)*nPOD )
    #print ('csv data - rows', t_begin+step ,'to', t_end+step )
    training_data[:,step*nPOD:(step+1)*nPOD] = csv_data[t_begin+step : t_end+step,:]

print('Shape of training data for the GAN', training_data.shape, training_data.dtype)

# GAN input
try:
    input_to_GAN = np.load('input_to_GAN.npy')
except:
    input_to_GAN = tf.random.normal([training_data.shape[0], ndims_latent_input])
    input_to_GAN = input_to_GAN.numpy()




Reading in the POD coeffs.
type and shape (nPOD by nTrain) of POD coeffs from csv file <class 'numpy.ndarray'> (5, 204) float32
shape csv data for the scaling (204, 5)
min and max of col,  0  of csv_data: -1.0 1.0
min and max of col,  1  of csv_data: -1.0 1.0000001
min and max of col,  2  of csv_data: -1.0 1.0
min and max of col,  3  of csv_data: -1.0 1.0
min and max of col,  4  of csv_data: -1.0 1.0
Shape of training data for the GAN (200, 25) float32


In [3]:
t0 = time.time()
predictions, generator = learn_hypersurface_from_POD_coeffs(input_to_GAN, training_data, nsteps, nPOD, ndims, lmbda, n_critic, batch_size, batches, ndims_latent_input)
t_train = time.time() - t0

looking for previous saved models
making new generator and critic
initialising logs for tensorboard
beginning training
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[ 0.36947453, -0.56368136,  0.42505252, -0.52954215, -0.3562048 ],
       [-0.09366822, -0.6652722 , -0.49555767,  0.11435276,  0.3177513 ],
       [ 0.35808337,  0.13187635, -0.26672226,  0.4327829 ,  0.251755  ],
       [-0.6485403 ,  0.14122897, -0.3009151 , -0.6499302 ,  0.4023627 ],
       [-0.20274329, -0.22054869,  0.4586861 ,  0.33503056,  0.45004296]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>]
epoch: 0 *************************************************
gen loss 0.5165217 d loss 5.405176 w_loss -0.20684215
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[ 0.36983585, -0.563494  ,  0.4244618 , -0.5294896 , -0.35570318],
       [-0.09364009, -0.6654967 , -0.49585912,  0.11403979,  0.

epoch: 11 *************************************************
gen loss 0.39308327 d loss 2.5052605 w_loss 0.19742014
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[ 0.36996797, -0.56202453,  0.4241141 , -0.5294824 , -0.35581687],
       [-0.09372365, -0.66739917, -0.49686006,  0.11142189,  0.31743345],
       [ 0.3608952 ,  0.13464211, -0.26752752,  0.43261847,  0.25207824],
       [-0.64642143,  0.14232738, -0.29954723, -0.65293723,  0.40147886],
       [-0.20197636, -0.2172606 ,  0.45820117,  0.3331667 ,  0.45036468]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=
array([-0.00011725, -0.00448398, -0.00075781, -0.00268445,  0.00027848],
      dtype=float32)>]
epoch: 12 *************************************************
gen loss 0.45525533 d loss 2.2407117 w_loss 0.23892185
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[ 0.37017402, -0.5623975 ,  0.42436108, -0.52967745, -0.35544986],
       [-0.09


KeyboardInterrupt

